<a href="https://colab.research.google.com/github/benab04/SalesForecasting/blob/main/SalesForecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install  pandas

In [2]:
%pip install -Uq upgini catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.3/151.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.3/108.3 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.2/433.2 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.3/109.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.6/277.6 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 913.9/913.9 kB 43.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

Prepare Input data


In [3]:
from os.path import exists
import pandas as pd


df_path="train.csv.zip" if exists("train.csv.zip") else "https://github.com/upgini/upgini/raw/main/notebooks/train.csv.zip"

df=pd.read_csv(df_path)
df = df.sample(n=19_000,random_state=0)

df["store"]=df["store"].astype(str)
df["item"]=df["item"].astype(str)

df["date"]=pd.to_datetime(df["date"])

df.sort_values("date",inplace=True)

df.reset_index(inplace=True, drop=True)

df.head()



,date,store,item,sales
0,2013-01-01,7,5,5
1,2013-01-01,4,9,19
2,2013-01-01,1,33,37
3,2013-01-01,3,41,14
4,2013-01-01,5,24,26


In [4]:
train=df[df["date"] <"2017-01-01"]
test=df[df["date"] >="2017-01-01"]


In [5]:
train_features=train.drop(columns=["sales"])
train_target=train["sales"]
test_features=train.drop(columns=["sales"])
test_target=train["sales"]

Enrich Data


In [6]:
from upgini import FeaturesEnricher,SearchKey
from upgini.metadata import CVType

enricher = FeaturesEnricher(
    search_keys={
        "date":SearchKey.DATE,
    },
    cv=CVType.time_series
)
enricher.fit(train_features,
             train_target,
             eval_set=[(test_features,test_target)]
             )

[==============================                              ] 50% Automated feature engineering...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

f_autofe_max_936,0.0563,100.0000,"0.7041, 1.0, 0.947",Upgini,AutoFE: features from Calendar data,Free
f_autofe_min_256,0.0543,100.0000,"-0.1501, -0.9416, -0.728",Upgini,AutoFE: features from Calendar data,Free
f_autofe_max_969,0.0211,100.0000,"0.9668, 0.9093, 0.5271",Upgini,AutoFE: features from Calendar data,Free
f_autofe_max_902,0.0168,100.0000,"0.8438, -0.0257, 0.9821",Upgini,AutoFE: features from Calendar data,Free
f_weather_date_weather_umap_34_c3ef5b4f,0.0167,100.0000,"5.1858, 5.2667, 4.8001",Upgini,Weather & climate normals data,Premium
f_autofe_max_1021,0.0059,100.0000,"0.112, 0.055, 0.56",Upgini,"AutoFE: features from Calendar data,Markets data",Free
f_autofe_min_276,0.0053,100.0000,"-0.734, 0.072, -0.044",Upgini,"AutoFE: features from Calendar data,Markets data",Free
f_autofe_div_1703,0.0043,100.0000,"-0.0087, -0.0189, -0.0239",Upgini,"AutoFE: features from Calendar data,Markets data",Free
f_autofe_div_1743,0.0031,100.0000,"-0.9589, -0.8993, -0.6612",Upgini,"AutoFE: features from Calendar data,Markets data",Free
f_events_date_year_cos1_9014a856,0.0027,100.0000,"-0.8945, 0.2261, 0.0429",Upgini,Calendar data,Free
f_autofe_min_262,0.0020,100.0000,"-0.2967, -0.9995, 0.6472",Upgini,AutoFE: features from Calendar data,Free


Upgini,AutoFE: features from Calendar data,0.1582,16
Upgini,"AutoFE: features from Calendar data,Markets data",0.0233,9
Upgini,Weather & climate normals data,0.0178,4
Upgini,AutoFE: features from Markets data,0.0029,9
Upgini,Calendar data,0.0027,1
Upgini,Markets data,0.0006,2
Upgini,AutoFE: feature from Markets data,0.0004,1
Upgini,World economic indicators,0.0003,2


"Calendar data,Markets data",f_autofe_div_1665,f_events_date_week_cos3_7525fe31,f_financial_date_crude_oil_7d_to_1y_c3e0ad17,div
"Calendar data,Markets data",f_autofe_div_1703,f_events_date_week_sin1_847b5db1,f_financial_date_crude_oil_1f195998,div
"Calendar data,Markets data",f_autofe_div_1741,f_events_date_year_cos1_9014a856,f_financial_date_crude_oil_1f195998,div
"Calendar data,Markets data",f_autofe_div_1743,f_events_date_year_cos1_9014a856,f_financial_date_crude_oil_7d_to_7d_1y_shift_718136ef,div
Markets data,f_autofe_div_2072,f_financial_date_crude_oil_7d_to_1y_c3e0ad17,f_financial_date_nasdaq_7d_to_7d_1y_shift_4a4dd6ce,div
Markets data,f_autofe_div_2136,f_financial_date_gold_1d_to_7d_742a0045,f_financial_date_stoxx_043cbcd4,div
Markets data,f_autofe_div_2210,f_financial_date_nasdaq_7d_to_7d_1y_shift_4a4dd6ce,f_financial_date_stoxx_7d_to_1y_2ea815f4,div
Markets data,f_autofe_div_2211,f_financial_date_nasdaq_7d_to_7d_1y_shift_4a4dd6ce,f_financial_date_stoxx_7d_to_7d_1y_shift_abef6e91,div
Markets data,f_autofe_div_2383,f_financial_date_snp500_7d_to_1y_378b9f99,f_financial_date_stoxx_043cbcd4,div
Markets data,f_autofe_div_2400,f_financial_date_stoxx_043cbcd4,f_financial_date_usd_7d_to_7d_1y_shift_35cb53e9,div
Calendar data,f_autofe_max_902,f_events_date_week_cos1_f6a8c1fc,f_events_date_year_cos1_9014a856,max


We detected 65 outliers in your sample.
Examples of outliers with maximum value of target:
47    196
38    187
12    171
Name: target, dtype: int64
Outliers will be excluded during the metrics calculation.
Before dropping target outliers size: 15213
After dropping target outliers size: 15148
Calculating accuracy uplift after enrichment...


Train,15213,49.9541,301.2609,183.9387,117.3222


In [7]:
from catboost import CatBoostRegressor
from catboost.utils import eval_metric

model=CatBoostRegressor(verbose=False, allow_writing_files=False, random_state=0)

enricher.calculate_metrics(
    train_features, train_target,
    eval_set =[(test_features, test_target)],
    estimator=model,
    scoring="mean_absolute_percentage_error"
)

Calculating accuracy uplift after enrichment...


,Dataset type,Rows,Mean target,Baseline mean_absolute_percentage_error,Enriched mean_absolute_percentage_error,Uplift
0,Train,15213,49.9541,0.25285,0.156026,0.096824


In [ ]:
# Example of batch processing
batch_size = 1000
enriched_train_features = pd.DataFrame()
enriched_test_features = pd.DataFrame()

for i in range(0, len(train_features), batch_size):
    batch = train_features.iloc[i:i + batch_size]
    enriched_batch = enricher.transform(batch, keep_input=True)
    # Instead of using frame.append, use pandas.concat
    enriched_train_features = pd.concat([enriched_train_features, enriched_batch], ignore_index=True)

for i in range(0, len(test_features), batch_size):
    batch = test_features.iloc[i:i + batch_size]
    enriched_batch = enricher.transform(batch, keep_input=True)
    enriched_test_features = pd.concat([enriched_test_features, enriched_batch], ignore_index=True)

enriched_train_features.head()

[============================================================] 100% Finished

You use Trial access to Upgini data enrichment. Limit for Trial: 10000 rows. You have already enriched: 5384 rows.

Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history


Column name,Status,Errors
date,All valid,-



Running search request, search_id=bbc3ba05-b652-487e-a757-3cc9be7c74fc
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com

Retrieving selected features from data sources...


[============================================================] 100% Finished

...


Running search request, search_id=33369f69-edde-4048-9df1-94a4a54a8dc8
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com

Retrieving selected features from data sources...


[=================================================           ] 82% Generating report...

You use Trial access to Upgini data enrichment. Limit for Trial: 10000 rows. You have already enriched: 6647 rows.

Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history


Column name,Status,Errors
date,All valid,-



Running search request, search_id=35e626ee-97f3-4546-b8f9-c65d88f1aa9f
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com


In [21]:
model.fit(train_features, train_target)
preds=model.predict(test_features)
eval_metric(test_target.values,preds,"SMAPE")

[18.295517483678655]

In [22]:
print("Shape of enriched_train_features:", enriched_train_features.shape)
print("Shape of train_target:", train_target.shape)

Shape of enriched_train_features: (15213, 30)
Shape of train_target: (15213,)


In [24]:
model.fit(enriched_train_features, train_target)
enriched_preds=model.predict(enriched_test_features)
eval_metric(test_target.values,enriched_preds,"SMAPE")

[13.90813069519919]